Hipótesis 1:
H0: No hay diferencia significativa en los retrasos de salida (DepDelay) entre las diferentes aerolíneas (Carrier).
Estadístico de prueba: ANOVA.

Hipótesis 2:
H0: No existe relación entre el día de la semana (DayOfWeek) y si un vuelo se retrasa más de 15 minutos en la llegada (ArrDelay15).
Estadístico de prueba: Chi-Cuadrada.

Hipótesis 3:
H0: No hay diferencia significativa en el retraso promedio de llegada (ArrDelay) entre los aeropuertos de origen (OriginAirportID).
Estadístico de prueba: ANOVA.

Hipótesis 4:
H0: El retraso promedio en la salida (DepDelay) es igual al retraso promedio en la llegada (ArrDelay).
Estadístico de prueba: t-student para muestras emparejadas.

Hipótesis 5:
H0: No hay diferencia significativa en los retrasos de llegada (ArrDelay) entre vuelos que despegan antes del mediodía (CRSDepTime < 1200) y los que despegan después del mediodía (CRSDepTime >= 1200).
Estadístico de prueba: t-student para muestras independientes.

Hipótesis 6:
H0: El porcentaje de vuelos cancelados (Cancelled) es independiente del mes del año (Month).
Estadístico de prueba: Chi-Cuadrada.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import chi2_contingency

# Cargar los datos
data = pd.read_csv('vuelos.csv')

# Hipótesis 1: ANOVA entre DepDelay y Carrier
anova_model1 = ols('DepDelay ~ C(Carrier)', data=data).fit()
anova_table1 = sm.stats.anova_lm(anova_model1, typ=2)
print("ANOVA Resultados - Hipótesis 1 (DepDelay vs Carrier):\n", anova_table1)

# Hipótesis 2: Chi-Cuadrada entre DayOfWeek y ArrDelay15
contingency_table2 = pd.crosstab(data['DayOfWeek'], data['ArrDelay15'])
chi2_2, p2, dof2, expected2 = chi2_contingency(contingency_table2)
print(f"Chi-Cuadrada - Hipótesis 2 (DayOfWeek vs ArrDelay15): chi2={chi2_2}, p-value={p2}")

# Hipótesis 3: ANOVA entre ArrDelay y OriginAirportID
anova_model3 = ols('ArrDelay ~ C(OriginAirportID)', data=data).fit()
anova_table3 = sm.stats.anova_lm(anova_model3, typ=2)
print("ANOVA Resultados - Hipótesis 3 (ArrDelay vs OriginAirportID):\n", anova_table3)

# Hipótesis 4: t-test entre DepDelay y ArrDelay
t_stat4, p_val4 = stats.ttest_rel(data['DepDelay'], data['ArrDelay'], nan_policy='omit')
print(f"t-test - Hipótesis 4 (DepDelay vs ArrDelay): t-stat={t_stat4}, p-value={p_val4}")

# Hipótesis 5: t-test entre vuelos antes y después del mediodía (CRSDepTime)
data['AM_PM'] = data['CRSDepTime'].apply(lambda x: 'AM' if x < 1200 else 'PM')
t_stat5, p_val5 = stats.ttest_ind(data[data['AM_PM'] == 'AM']['ArrDelay'].dropna(), 
                                  data[data['AM_PM'] == 'PM']['ArrDelay'].dropna(), 
                                  equal_var=False)
print(f"t-test - Hipótesis 5 (AM vs PM DepTime): t-stat={t_stat5}, p-value={p_val5}")

# Hipótesis 6: Chi-Cuadrada entre Cancelled y Month
contingency_table6 = pd.crosstab(data['Month'], data['Cancelled'])
chi2_6, p6, dof6, expected6 = chi2_contingency(contingency_table6)
print(f"Chi-Cuadrada - Hipótesis 6 (Month vs Cancelled): chi2={chi2_6}, p-value={p6}")

# Gráficos
# 1. Boxplot para Hipótesis 1 (DepDelay vs Carrier)
plt.figure(figsize=(10,6))
sns.boxplot(x='Carrier', y='DepDelay', data=data)
plt.title('Distribución de Retrasos por Aerolínea')
plt.show()

# 2. Heatmap para Hipótesis 2 (DayOfWeek vs ArrDelay15)
sns.heatmap(contingency_table2, annot=True, cmap="YlGnBu")
plt.title('Heatmap de Retrasos en Llegada por Día de la Semana')
plt.show()

# 3. Boxplot para Hipótesis 3 (ArrDelay vs OriginAirportID)
plt.figure(figsize=(10,6))
sns.boxplot(x='OriginAirportID', y='ArrDelay', data=data)
plt.title('Distribución de Retrasos por Aeropuerto de Origen')
plt.show()

# 4. Histograma comparativo para Hipótesis 4 (DepDelay vs ArrDelay)
plt.figure(figsize=(10,6))
plt.hist(data['DepDelay'], bins=50, alpha=0.5, label='DepDelay')
plt.hist(data['ArrDelay'], bins=50, alpha=0.5, label='ArrDelay')
plt.title('Comparación de Retrasos en Salida y Llegada')
plt.legend()
plt.show()

# 5. Boxplot para Hipótesis 5 (Retrasos AM vs PM)
plt.figure(figsize=(10,6))
sns.boxplot(x='AM_PM', y='ArrDelay', data=data)
plt.title('Comparación de Retrasos en Vuelos AM vs PM')
plt.show()

# 6. Heatmap para Hipótesis 6 (Month vs Cancelled)
sns.heatmap(contingency_table6, annot=True, cmap="YlGnBu")
plt.title('Heatmap de Vuelos Cancelados por Mes')
plt.show()